In [52]:
import os
import pandas as pd
import lyricsgenius as lg
import spotipy
import time

In [53]:
spotipy_client_id = 'c3e43077142f4e7bb70601747bd1f0d2'
spotipy_client_secret = '925f7025c8e6420f9be665401784c0f6'
spotipy_redirect_uri = 'https://google.com'
genius_access_token = 'jVHnDFkBvrvEsj_M8YUxbQ9t8mo2zIG19giBxlX_g7NeRz4W51gedM1YHTBX_U-4'

In [54]:
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials (spotipy_client_id, spotipy_client_secret)
sp=spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_playlist_tracks(playlist_id):
    uri = []
    track = []
    artists = []
    details = sp.playlist_tracks(playlist_id)
    df1 = pd.DataFrame(details)
    
    for i in range(len(df1['items'])):
        uri.append(df1['items'][i]['track']['uri'])
        track.append(df1['items'][i]['track']['name'])
        artists.append(df1['items'][i]['track']['artists'][0]['name'])
    
    df2 = pd.DataFrame({
    'uri':uri,
    'track':track,
    'artists':artists})
    
    return df2

In [55]:
playlist_id = input('Enter the playlist link')
playlist_id = playlist_id[34:56]
tracks_df  = get_playlist_tracks(playlist_id)


In [56]:
tracks_df

,uri,track,artists
0,spotify:track:4UG2Fm0E98LhE2dlNxiAXx,this is what falling in love feels like,JVKE
1,spotify:track:0qXP5fMhxGzxALOkXYUxfP,Loverboy,A-Wall
2,spotify:track:27rZYxE4l21wTaovX4WTnI,Television / So Far So Good,Rex Orange County
3,spotify:track:0sTlGEld0h8kIPZaKDYUf4,Notion,The Rare Occasions
4,spotify:track:1eFYbcVZBTKcE0iFIYU5qk,Not Into You,Brooksie
5,spotify:track:4SqWKzw0CbA05TGszDgMlc,I Love You So,The Walters
6,spotify:track:0DGTcMqvVR7fmBXgiG6jz4,Just A Cloud Away,Pharrell Williams
7,spotify:track:61YzdCCBPM5Pc7lIiD5i8C,Bang Bang,K'NAAN
8,spotify:track:5o5WaO9pzPhXSGIrTIYVce,Cherry Wine,grentperez
9,spotify:track:294kJwjWXPTgTD1MNM1crh,Lay,SOL7DER


In [57]:
#insert output dataframe from the get_album_tracks function
def get_track_info(df):
    danceability = []
    energy = []
    key = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    for i in df['uri']:
        for x in sp.audio_features(tracks=[i]):
            danceability.append(x['danceability'])
            energy.append(x['energy'])
            key.append(x['key'])
            loudness.append(x['loudness'])
            speechiness.append(x['speechiness'])
            acousticness.append(x['acousticness'])
            instrumentalness.append(x['instrumentalness'])
            liveness.append(x['liveness'])
            valence.append(x['valence'])
            tempo.append(x['tempo'])
            
    df2 = pd.DataFrame({
    'danceability':danceability,
    'energy':energy,
    'key':key,
    'loudness':loudness,
    'speechiness':speechiness,
    'acousticness':acousticness,
    'instrumentalness':instrumentalness,
    'liveness':liveness,
    'valence':valence,
    'tempo':tempo})
    
    return df2

In [58]:
features_df = get_track_info(tracks_df) 

In [59]:
features_df

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.424,0.438,11,-5.530,0.0553,0.6170,0.000000,0.0837,0.325,128.983
1,0.604,0.466,0,-10.093,0.0315,0.6490,0.033400,0.0930,0.499,120.045
2,0.403,0.505,5,-7.750,0.0463,0.5920,0.000000,0.1210,0.320,164.902
3,0.309,0.883,9,-3.825,0.0808,0.0673,0.001110,0.0849,0.312,159.488
4,0.641,0.678,3,-4.943,0.0355,0.2210,0.573000,0.2790,0.495,100.035
5,0.575,0.648,10,-4.891,0.0358,0.5830,0.000000,0.1150,0.466,75.977
6,0.890,0.460,2,-8.155,0.0359,0.4320,0.000000,0.1100,0.677,112.555
7,0.623,0.901,10,-4.402,0.1820,0.0785,0.000000,0.0251,0.896,179.985
8,0.740,0.563,11,-7.016,0.0400,0.3400,0.000000,0.0824,0.577,104.958
9,0.606,0.817,8,-6.512,0.0500,0.0216,0.000000,0.0466,0.806,90.343


In [60]:
main_df = tracks_df.merge(features_df,left_index=True,right_index=True)

In [61]:
main_df

,uri,track,artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,spotify:track:4UG2Fm0E98LhE2dlNxiAXx,this is what falling in love feels like,JVKE,0.424,0.438,11,-5.530,0.0553,0.6170,0.000000,0.0837,0.325,128.983
1,spotify:track:0qXP5fMhxGzxALOkXYUxfP,Loverboy,A-Wall,0.604,0.466,0,-10.093,0.0315,0.6490,0.033400,0.0930,0.499,120.045
2,spotify:track:27rZYxE4l21wTaovX4WTnI,Television / So Far So Good,Rex Orange County,0.403,0.505,5,-7.750,0.0463,0.5920,0.000000,0.1210,0.320,164.902
3,spotify:track:0sTlGEld0h8kIPZaKDYUf4,Notion,The Rare Occasions,0.309,0.883,9,-3.825,0.0808,0.0673,0.001110,0.0849,0.312,159.488
4,spotify:track:1eFYbcVZBTKcE0iFIYU5qk,Not Into You,Brooksie,0.641,0.678,3,-4.943,0.0355,0.2210,0.573000,0.2790,0.495,100.035
5,spotify:track:4SqWKzw0CbA05TGszDgMlc,I Love You So,The Walters,0.575,0.648,10,-4.891,0.0358,0.5830,0.000000,0.1150,0.466,75.977
6,spotify:track:0DGTcMqvVR7fmBXgiG6jz4,Just A Cloud Away,Pharrell Williams,0.890,0.460,2,-8.155,0.0359,0.4320,0.000000,0.1100,0.677,112.555
7,spotify:track:61YzdCCBPM5Pc7lIiD5i8C,Bang Bang,K'NAAN,0.623,0.901,10,-4.402,0.1820,0.0785,0.000000,0.0251,0.896,179.985
8,spotify:track:5o5WaO9pzPhXSGIrTIYVce,Cherry Wine,grentperez,0.740,0.563,11,-7.016,0.0400,0.3400,0.000000,0.0824,0.577,104.958
9,spotify:track:294kJwjWXPTgTD1MNM1crh,Lay,SOL7DER,0.606,0.817,8,-6.512,0.0500,0.0216,0.000000,0.0466,0.806,90.343


#### Genius API to fetch lyrics

In [62]:
genius = lg.Genius(genius_access_token)

In [63]:
l=[]
for i in range(len(main_df['track'])):
  song = genius.search_song(title=main_df['track'][i],artist=main_df['artists'][i])
  lyrics = song.lyrics
  if song == None:
    l.append(None)
  else:
    l.append(lyrics)

lyrics_df = pd.DataFrame()
lyrics_df['lyrics'] = l

Searching for "this is what falling in love feels like" by JVKE...
Done.
Searching for "Loverboy" by A-Wall...
Done.
Searching for "Television / So Far So Good" by Rex Orange County...
Done.
Searching for "Notion" by The Rare Occasions...
Done.
Searching for "Not Into You" by Brooksie...
Done.
Searching for "I Love You So" by The Walters...
Done.
Searching for "Just A Cloud Away" by Pharrell Williams...
Done.
Searching for "Bang Bang" by K'NAAN...
Done.
Searching for "Cherry Wine" by grentperez...
Done.
Searching for "Lay" by SOL7DER...
Done.
Searching for "anxiety." by JVKE...
Done.
Searching for "this is what falling in love feels like - Leon Leiden Remix" by JVKE...
Done.
Searching for "this is what falling in love feels like - Leon Leiden Remix" by JVKE...
Done.
Searching for "Nuestra Canción" by Monsieur Periné...
Done.
Searching for "last day on earth" by Tai Verdes...
Done.


In [64]:
# join both data frames
main_df = pd.concat([main_df, lyrics_df], axis=1, join='inner')
main_df.head(1)

,uri,track,artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics
0,spotify:track:4UG2Fm0E98LhE2dlNxiAXx,this is what falling in love feels like,JVKE,0.424,0.438,11,-5.53,0.0553,0.617,0.0,0.0837,0.325,128.983,​this is what falling in love feels like Lyric...


## DATA PRE-PROCESSING

### 1.Data Cleaning

#### Checking for missing values

In [65]:
main_df.isna().sum()

uri                 0
track               0
artists             0
danceability        0
energy              0
key                 0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
lyrics              0
dtype: int64

#### Checking for Duplicate values

In [66]:
main_df['track'].duplicated().sum()

1

In [67]:
main_df.drop_duplicates(subset='track',keep='last',inplace=True)

### <b>2.Lower Casing</b>

In [69]:
main_df['lyrics'] = main_df['lyrics'].apply(lambda x:x.lower())

### <b>3.Tokenization</b>

In [74]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91635\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [72]:
from nltk.tokenize import RegexpTokenizer
Regtokenizer = RegexpTokenizer(main_df['lyrics'])
print(Regtokenizer)

RegexpTokenizer(pattern=0     ​this is what falling in love feels like lyric...
1     loverboy lyrics[verse 1]\nyeah you can call me...
2     television / so far so good lyrics[part i: tel...
3     notion lyrics[verse 1]\nsure it’s a calming no...
4     not into you lyrics[intro]\ndude, she's just n...
5     translationsрусскийi love you so lyrics[verse ...
6     just a cloud away lyrics[chorus: pharrell will...
7     bang bang lyrics[hook: k'naan]\nshe shot me, s...
8     cherry wine lyrics[verse 1]\nwhy you standing ...
9     celtic boast battle lyricsint. camp\nthere's a...
10    ​anxiety. lyrics[verse 1]\nyeah, i’ve been str...
12    novedades de viernes 06/05/22 lyrics* = cancio...
13    nuestra canción lyrics[letra de "nuestra canci...
14    last day on earth lyrics[chorus]\nif it was my...
Name: lyrics, dtype: object, gaps=False, discard_empty=True, flags=re.UNICODE|re.MULTILINE|re.DOTALL)


In [76]:

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

main_df['lyrics'] = main_df['lyrics'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

main_df['lyrics'] = main_df['lyrics'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

main_df['lyrics'] = main_df['lyrics'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

main_df['lyrics'] = main_df['lyrics'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

main_df['lyrics'] = main_df['lyrics'].apply(remove_singl_char)

# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

main_df['lyrics_tokens']= main_df['lyrics'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(main_df['lyrics_tokens'])

Tokenizing Result : 

0     [​this, is, what, falling, in, love, feels, li...
1     [loverboy, lyricsverse, yeah, you, can, call, ...
2     [television, so, far, so, good, lyricspart, te...
3     [notion, lyricsverse, sure, it, ’, calming, no...
4     [not, into, you, lyricsintro, dude, shes, just...
5     [translationsрусскийi, love, you, so, lyricsve...
6     [just, cloud, away, lyricschorus, pharrell, wi...
7     [bang, bang, lyricshook, knaan, she, shot, me,...
8     [cherry, wine, lyricsverse, why, you, standing...
9     [celtic, boast, battle, lyricsint, camp, there...
10    [​anxiety, lyricsverse, yeah, ’, ve, been, str...
12    [novedades, de, viernes, lyrics, canciones, in...
13    [nuestra, canción, lyricsletra, de, nuestra, c...
14    [last, day, on, earth, lyricschorus, if, it, w...
Name: lyrics_tokens, dtype: object


### <b>4.Stopwords Removal</b>

In [78]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91635\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [83]:
# Stopwords Removal
from nltk.corpus import stopwords

stop = stopwords.words('english')
main_df['lyrics_tokens_preprocessing'] = main_df['lyrics_tokens'].apply(lambda x: [item for item in x if item not in stop])

### <b>5.Lemmatization</b>

In [87]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91635\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\91635\AppData\Roaming\nltk_data...


True

In [88]:
# Lemmatization
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

main_df['lyrics_tokens_preprocessing'] = main_df['lyrics_tokens_preprocessing'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

### <b>6.Stemming</b>

In [89]:
# stemming
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
main_df['lyrics_tokens_preprocessing'] = main_df['lyrics_tokens_preprocessing'].apply(lambda x: [stemmer.stem(y) for y in x])

In [90]:
main_df.head()

,uri,track,artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics,lyrics_tokens,Lyrics_tokens_preprocessing,lyrics_tokens_preprocessing
0,spotify:track:4UG2Fm0E98LhE2dlNxiAXx,this is what falling in love feels like,JVKE,0.424,0.438,11,-5.530,0.0553,0.6170,0.00000,0.0837,0.325,128.983,​this is what falling in love feels like lyric...,"[​this, is, what, falling, in, love, feels, li...","[​this, falling, love, feels, like, lyricschor...","[​thi, fall, love, feel, like, lyricschoru, fe..."
1,spotify:track:0qXP5fMhxGzxALOkXYUxfP,Loverboy,A-Wall,0.604,0.466,0,-10.093,0.0315,0.6490,0.03340,0.0930,0.499,120.045,loverboy lyricsverse yeah you can call me love...,"[loverboy, lyricsverse, yeah, you, can, call, ...","[loverboy, lyricsverse, yeah, call, lover, boy...","[loverboy, lyricsvers, yeah, call, lover, boy,..."
2,spotify:track:27rZYxE4l21wTaovX4WTnI,Television / So Far So Good,Rex Orange County,0.403,0.505,5,-7.750,0.0463,0.5920,0.00000,0.1210,0.320,164.902,television so far so good lyricspart televisio...,"[television, so, far, so, good, lyricspart, te...","[television, far, good, lyricspart, television...","[televis, far, good, lyricspart, televis, vers..."
3,spotify:track:0sTlGEld0h8kIPZaKDYUf4,Notion,The Rare Occasions,0.309,0.883,9,-3.825,0.0808,0.0673,0.00111,0.0849,0.312,159.488,notion lyricsverse sure it’ calming notion per...,"[notion, lyricsverse, sure, it, ’, calming, no...","[notion, lyricsverse, sure, ’, calming, notion...","[notion, lyricsvers, sure, ’, calm, notion, pe..."
4,spotify:track:1eFYbcVZBTKcE0iFIYU5qk,Not Into You,Brooksie,0.641,0.678,3,-4.943,0.0355,0.2210,0.57300,0.2790,0.495,100.035,not into you lyricsintro dude shes just not in...,"[not, into, you, lyricsintro, dude, shes, just...","[lyricsintro, dude, shes, got, ta, move, move,...","[lyricsintro, dude, she, got, ta, move, move, ..."


### VADER ALGO

In [31]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

sentiment_list = []
sentiment_score_list = []

for i in main_df['lyrics']:
    try:
        sentiment_score = analyser.polarity_scores(i)

        if sentiment_score['compound'] >= 0.05:
            sentiment_percentage = sentiment_score['compound']
            sentiment = 'Positive'
        elif sentiment_score['compound'] > -0.05 and sentiment_score['compound'] < 0.05:
            sentiment_percentage = sentiment_score['compound']
            sentiment = 'Neutral'
        elif sentiment_score['compound'] <= -0.05:
            sentiment_percentage = sentiment_score['compound']
            sentiment = 'Negative'

        sentiment_list.append(sentiment)
        sentiment_score_list.append((abs(sentiment_percentage) * 100))
        
    except:
        sentiment_list.append('None')
        sentiment_score_list.append(0)

main_df['Sentiment'] = sentiment_list
main_df['Sentiment_Score'] = sentiment_score_list


In [32]:
main_df

,uri,track,artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics,Sentiment,Sentiment_Score
0,spotify:track:4UG2Fm0E98LhE2dlNxiAXx,this is what falling in love feels like,JVKE,0.424,0.438,11,-5.530,0.0553,0.6170,0.000000,0.0837,0.325,128.983,​this is what falling in love feels like Lyric...,Positive,99.33
1,spotify:track:0qXP5fMhxGzxALOkXYUxfP,Loverboy,A-Wall,0.604,0.466,0,-10.093,0.0315,0.6490,0.033400,0.0930,0.499,120.045,Loverboy Lyrics[Verse 1]\nYeah you can call me...,Positive,99.50
2,spotify:track:27rZYxE4l21wTaovX4WTnI,Television / So Far So Good,Rex Orange County,0.403,0.505,5,-7.750,0.0463,0.5920,0.000000,0.1210,0.320,164.902,Television / So Far So Good Lyrics[Part I: Tel...,Negative,88.96
3,spotify:track:0sTlGEld0h8kIPZaKDYUf4,Notion,The Rare Occasions,0.309,0.883,9,-3.825,0.0808,0.0673,0.001110,0.0849,0.312,159.488,Notion Lyrics[Verse 1]\nSure it’s a calming no...,Positive,98.67
4,spotify:track:1eFYbcVZBTKcE0iFIYU5qk,Not Into You,Brooksie,0.641,0.678,3,-4.943,0.0355,0.2210,0.573000,0.2790,0.495,100.035,"Not Into You Lyrics[Intro]\nDude, she's just n...",Positive,69.87
5,spotify:track:4SqWKzw0CbA05TGszDgMlc,I Love You So,The Walters,0.575,0.648,10,-4.891,0.0358,0.5830,0.000000,0.1150,0.466,75.977,TranslationsРусскийI Love You So Lyrics[Verse ...,Positive,99.74
6,spotify:track:0DGTcMqvVR7fmBXgiG6jz4,Just A Cloud Away,Pharrell Williams,0.890,0.460,2,-8.155,0.0359,0.4320,0.000000,0.1100,0.677,112.555,Just a Cloud Away Lyrics[Chorus: Pharrell Will...,Positive,98.63
7,spotify:track:61YzdCCBPM5Pc7lIiD5i8C,Bang Bang,K'NAAN,0.623,0.901,10,-4.402,0.1820,0.0785,0.000000,0.0251,0.896,179.985,"Bang Bang Lyrics[Hook: K'naan]\nShe shot me, s...",Positive,99.75
8,spotify:track:5o5WaO9pzPhXSGIrTIYVce,Cherry Wine,grentperez,0.740,0.563,11,-7.016,0.0400,0.3400,0.000000,0.0824,0.577,104.958,Cherry Wine Lyrics[Verse 1]\nWhy you standing ...,Positive,91.43
9,spotify:track:294kJwjWXPTgTD1MNM1crh,Lay,SOL7DER,0.606,0.817,8,-6.512,0.0500,0.0216,0.000000,0.0466,0.806,90.343,Celtic Boast Battle LyricsINT. CAMP\nThere's a...,Negative,99.84


for i in main_df['lyrics']:
  print(i)
  print('---------------------------------------------')